# Adult Census Income - A classification problem
Using income data of the adult US population, we want to classify who earns more or less than 50k USD

## Prerequirements

General imports needed

In [ ]:
import time
import pandas as pd
import logging as logging
import pprint
import plotly
plotly.offline.init_notebook_mode(connected=True)

Pailab specific imports

In [ ]:
from pailab import MLRepo, MeasureConfiguration, FIRST_VERSION, LAST_VERSION

Basic configuration

In [ ]:
logging.basicConfig(level=logging.FATAL)

## Read the data

In this notebook we use the adult data from the census 1994 it can be found in [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) or from [Kaggle](https://www.kaggle.com/uciml/adult-census-income). The data set consists of 48842 observations each with 14 attributes. The table consists of qualitative (e.g. workclass, education) and quantitative attributes.

In [ ]:
data = pd.read_csv('adult.csv')

Some basic adjustments, which are not part of the preprocessing

In [ ]:
# change the output format
data['income']=data['income'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

convert categorical features into dummy variable

In [ ]:
categorical_columns = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
data = pd.get_dummies(data, columns=categorical_columns)

# Create a new repository
We first create a new repository for our task. The repository is the central key around all functionality is built. Similar to a repository used for source control in classical software development it contains all data and algorithms needed for the machine learning task. The repository needs storages for 
- scripts containing the machine learning algorithms and interfaces,
- numerical objects such as arrays and matrices representing data, e.g. input data, data from the valuation of the models,
- json documents representing parameters, e.g. training parameter, model parameter.

To keep things simple, we just start using in memory storages. Note that the used memory interfaces are except for testing and playing around not be the first choice, since when ending the session, everything will be lost...

In addition to the storages the repository needs a reference to a JobRunner which the platform can use to execute machine learning jobs. For this example we use the most simple one, executing everything sequential in the same thread, the repository runs in.

In [ ]:
# setting up the repository
config = None

# set this to True if you would like to test the git_handler together with a 
# hdf handler for the big data parts instead of the memory handler
if False: 
    config = {'user': 'test_user',
          'workspace': 'c:/temp',
          'repo_store': 
          {
              'type': 'git_handler',  
              'config': {
                  'folder': 'c:/temp', 
                  'file_format': 'pck'
              }
          },
          'numpy_store':
          {
              'type': 'hdf_handler',
              'config':{
                  'folder': 'c:/temp/hdf',
                  'version_files': True
              }
          }
         }

ml_repo = MLRepo( user = 'test_user', config=config)
job_runner = ml_repo._job_runner

## Add a tree
To navigate in a simple way over all objects, one can add a so-called tree to the repository. The tree allows one to use auto completion to acces objcts and respectiv methods.

In [ ]:
from pailab.tools.tree import MLTree
MLTree.add_tree(ml_repo)

## Adding data
The data in the repository is handled by two different data objects:
- RawData is the object containing real data.
- DataSet is the object conaining the logical data, i.e. a reference to a RawData object together with a specification, which data from the RawData will be used. Here, one can specify a fixed version of the underlying RawData object (then changes to the RawData will not affect the derived DataSet) or a fixed or floating subset of the RawData by defining start and endindex cutting the derived data just out of the original data.

Normally one will add RawData and then define DataSets which are used to train or test a model which is exactly the way shown in the following. 
The target variable is 'income'. By default all other variables are used for as input_variables.

In [ ]:
ml_repo.tree.raw_data.add('adult_census_data', data, target_variables=['income'])

In [ ]:
# Add RawData. A convenient way to add RawData is simply to use the method add on the raw_data collection.
# This method just takes a pandas dataframe and the specification, which columns belong to the input 
#and which to the targets.
ml_repo.tree.raw_data.add('adult_census_data', data, target_variables = ['income'])
# based on the raw data we now define training and test sets
ml_repo.tree.training_data.add('training_sample', ml_repo.tree.raw_data.adult_census_data(), 0, 1000)
ml_repo.tree.test_data.add('test_sample', ml_repo.tree.raw_data.adult_census_data(), 1001, None)

In [ ]:
ml_repo.tree.training_data.training_sample.history()

When creating the DataSet we have to set two important informations for the repository, given as a dictionary:
- The object name. Each object in the repository needs to have a unique name in the repository.
- The object type which gives. In our example here we say that we specify that the DataSet are training and test data. Note that on can have only one training data object pre repository while the repository can obtain many different test data sets.

Some may wonder what is now stored in *version_list*.
** Adding an object (independent if it is a data object or some other object such as a parameter), the object gets a version number and no object will be removed, adding just adds a new version.** The add method returns a dictionary of the object names together with their version number.

## Adding a model
The next step to do machine learning would be to define a model which will be used in the repository. A model consists of the following pieces
- a skript where the code for the model valuation is defined together with the function name of the evaluation method
- a skript where the code for the model training is defined together with th function nam of the training method
- a model parameter object defining the model parameter and which must have implemented the correct interface so that it can be used within the repository (see the documentation on integrating new objects, normally there is not more to do then just simply add *@repo_object_init()* to the line above your *__init__* method)
- a training parameter object defining training parameters (such as number of optimization steps etc.), if necessary for your algorithms (this oen is optional)

** SKLearn models as an example**

We do not have to define the pieces defined above, if we use the sklearn module. Instead we can use the pailab.externals.sklearn module interfacing 
the sklearn package so that this can be used within the repository. This interface provides a simple method (add_model) to add an arbitrary sklearn model as a model which can be handled by the repository. This method adds a bunch of repo objects to the repository (according to the pieces described above):
- An object defining the function to be called to evaluate the model
- An object defining the function to be called to train the model
- An object defining the model
- An object defining the model parameter
For the following we just use a DecisionTree as our model.

### First add the preprocessing

In [ ]:
# import pailab.externals.pandas_interface as pandas_interface

# categorical_columns = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
# pandas_interface.add_preprocessor(ml_repo, pd.get_dummies, preprocessor_name='Pandas_Get_Dummies', preprocessor_param={columns=categorical_columns})

In [ ]:
import pailab.externals.numpy_interface as numpy_interface
select_columns = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week', 'workclass_?','workclass_Local-gov', 'workclass_Private', 'workclass_Self-emp-not-inc', 'education_HS-grad', 'education_Some-college', 'education_Bachelors', 'education_Masters',
'marital.status_Divorced', 'marital.status_Married-civ-spouse', 'marital.status_Never-married', 'marital.status_Separated', 
'occupation_Prof-specialty', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Adm-clerical', 'relationship_Husband', 'relationship_Not-in-family', 'relationship_Own-child', 'relationship_Unmarried', 'race_Asian-Pac-Islander', 'race_Black', 'race_White', 'sex_Female', 
'native.country_United-States', 'native.country_Mexico', 'native.country_?', 'native.country_Philippines']
numpy_interface.add_preprocessor_select_columns(ml_repo, preprocessor_name='Numpy_Select_Columns', 
                                                preprocessor_param={'columns':select_columns})

In [ ]:
import pailab.externals.sklearn_interface as sklearn_interface
from sklearn.preprocessing import StandardScaler

sklearn_interface.add_preprocessor(ml_repo, StandardScaler(), preprocessor_name='SKLStandardScaler')

### Add the model

In [ ]:
from sklearn.linear_model import LogisticRegression
sklearn_interface.add_model(ml_repo, LogisticRegression(), preprocessors=['Numpy_Select_Columns', 'SKLStandardScaler'], model_param={'max_iter': 1000})

## Train the model
Now, model taining is very simple, since you have defined training and testing data as well as  methods to value and fit your model and the model parameter.
So, you can just call *run_training* on the repository, and the training is perfomred automatically.
The training job is executed via the JobRunner you specified setting up the repository. All method of the repository involving jobs return the job id when adding the job to the JobRunner so that you can control the status of the task and see if it sucessfully finished.

In [ ]:
job_id = ml_repo.run_training() 
print(job_id)
job_info = job_runner.get_info(job_id[0], job_id[1])

## Run evaluation
To measure errors and to provide plots the model must be evaluated on all test and training datasets.

In [ ]:
ml_repo.tree.models.LogisticRegression.model.load()


In [ ]:
ml_repo.tree.models.LogisticRegression.model.obj.to_dict()

In [ ]:
job_id = ml_repo.run_evaluation()
# print information about the job
info = job_runner.get_info(job_id[0][0], job_id[0][1]) 
print(str(info))

## Add and compute measures

In [ ]:
ml_repo.add_measure(MeasureConfiguration.MAX)
ml_repo.add_measure(MeasureConfiguration.R2)

In [ ]:
job_ids = ml_repo.run_measures()

In [ ]:
ml_repo.tree.models.LogisticRegression.measures.test_sample.r2.load()
print(str(ml_repo.tree.models.LogisticRegression.measures.test_sample.r2.obj.value))

# Changing the preprocessor
Introducing a new version of the preprocessor to the repository. 

In [ ]:
sklearn_interface.add_preprocessor(
    ml_repo, StandardScaler(), preprocessor_name='SKLStandardScaler', preprocessor_param={'with_mean': False})

After we have changed the model parameter, we use the *checker* submodules *run* method for open tasks/inconsistencies. This method is called for the repository. If nothing is specified, all labeled models will be checked.
Applying the method to the latest model we see that the output shows that the models last version has been calibrated using a different model parameter version then the current version. 

In [ ]:
import pailab.tools.checker as checker
results = checker.run(ml_repo)
pprint.pprint(results)

In [ ]:
job_id = ml_repo.run_training(run_descendants=True)# we use run_descendants so that the issues with th measures are resolved too
print(job_id)

In [ ]:
results = checker.run(ml_repo)
pprint.pprint(results)